In [22]:
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn import model_selection
from sklearn import ensemble
#from sklearn_pandas import CategoricalImputer
from mlxtend import classifier as mlxClassifier #For Stacking

In [10]:
df = pd.read_csv('train.csv')

In [11]:
df.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color,type
0,0,0.354512,0.350839,0.465761,0.781142,clear,Ghoul
1,1,0.575560,0.425868,0.531401,0.439899,green,Goblin
2,2,0.467875,0.354330,0.811616,0.791225,black,Ghoul
3,4,0.776652,0.508723,0.636766,0.884464,black,Ghoul
4,5,0.566117,0.875862,0.418594,0.636438,green,Ghost


In [12]:
df.corr()

,id,bone_length,rotting_flesh,hair_length,has_soul
id,1.000000,-0.062759,-0.007539,0.076475,0.100054
bone_length,-0.062759,1.000000,-0.041716,0.353881,0.381675
rotting_flesh,-0.007539,-0.041716,1.000000,-0.220353,-0.132051
hair_length,0.076475,0.353881,-0.220353,1.000000,0.474835
has_soul,0.100054,0.381675,-0.132051,0.474835,1.000000


In [13]:
df.describe()

,id,bone_length,rotting_flesh,hair_length,has_soul
count,371.000000,371.000000,371.000000,371.000000,371.000000
mean,443.676550,0.434160,0.506848,0.529114,0.471392
std,263.222489,0.132833,0.146358,0.169902,0.176129
min,0.000000,0.061032,0.095687,0.134600,0.009402
25%,205.500000,0.340006,0.414812,0.407428,0.348002
50%,458.000000,0.434891,0.501552,0.538642,0.466372
75%,678.500000,0.517223,0.603977,0.647244,0.600610
max,897.000000,0.817001,0.932466,1.000000,0.935721


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 7 columns):
id               371 non-null int64
bone_length      371 non-null float64
rotting_flesh    371 non-null float64
hair_length      371 non-null float64
has_soul         371 non-null float64
color            371 non-null object
type             371 non-null object
dtypes: float64(4), int64(1), object(2)
memory usage: 17.4+ KB


In [15]:
df= pd.get_dummies(df,columns=['color'])

In [16]:

for i,row in df.iterrows():
    #print(i == 'Ghoul')
    #print(i)
    #print(row['type'])
    type_int=0
    if row['type'] == 'Ghoul':
        type_int=int(1)
    elif row['type'] == 'Goblin':
        type_int=int(2)
    else:
        type_int=int(3)
    df.at[i,'type_int'] = int(type_int)
    #break
df['type_int'] = pd.to_numeric(df['type_int'],downcast='integer')

In [17]:
df.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,type,color_black,color_blood,color_blue,color_clear,color_green,color_white,type_int
0,0,0.354512,0.350839,0.465761,0.781142,Ghoul,0,0,0,1,0,0,1
1,1,0.575560,0.425868,0.531401,0.439899,Goblin,0,0,0,0,1,0,2
2,2,0.467875,0.354330,0.811616,0.791225,Ghoul,1,0,0,0,0,0,1
3,4,0.776652,0.508723,0.636766,0.884464,Ghoul,1,0,0,0,0,0,1
4,5,0.566117,0.875862,0.418594,0.636438,Ghost,0,0,0,0,1,0,3


### Stacking

### Using Decission Tree, Random Forest, Ada Boosting

In [19]:
#build stacked model using selected features
rf1 = ensemble.RandomForestClassifier(random_state=100)
gb2 = ensemble.GradientBoostingClassifier(random_state=100)

dt = tree.DecisionTreeClassifier(random_state=100) #For Stacking

In [23]:
stacking_estimator = mlxClassifier.StackingClassifier(classifiers = [rf1,gb2], meta_classifier=dt)

In [38]:
stack_grid = {'randomforestclassifier__n_estimators': [5, 10],
              'randomforestclassifier__max_features': [4, 5, 6],
              'gradientboostingclassifier__n_estimators': [5, 10],
              'meta-decisiontreeclassifier__min_samples_split': [8, 9]}

In [39]:
#, 'has_soul'
X_train =df[[ 'bone_length', 'rotting_flesh', 'hair_length','has_soul',
       'color_black', 'color_blood', 'color_blue', 'color_clear',
       'color_green', 'color_white']]
Y_train = df['type_int']

In [40]:
grid_stack_estimator = model_selection.GridSearchCV(stacking_estimator, stack_grid, cv=10)
grid_stack_estimator.fit(X_train, Y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=StackingClassifier(average_probas=False,
          classifiers=[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_...ures=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'randomforestclassifier__n_estimators': [5, 10], 'randomforestclassifier__max_features': [4, 5, 6], 'gradientboostingclassifier__n_estimators': [5, 10], 'meta-decisiontreeclassifier__min_samples_split': [8, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [41]:
print(grid_stack_estimator.best_score_)
print(grid_stack_estimator.best_params_)

0.7035040431266847
{'gradientboostingclassifier__n_estimators': 5, 'meta-decisiontreeclassifier__min_samples_split': 8, 'randomforestclassifier__max_features': 5, 'randomforestclassifier__n_estimators': 10}


In [42]:
print(grid_stack_estimator.score(X_train, Y_train))

0.9892183288409704


In [30]:
final_model = grid_stack_estimator.best_estimator_
print(final_model.clfs_) #Classifiers
print(final_model.meta_clf_) #Meta Classifiers

[RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=7, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=100, verbose=0, warm_start=False), GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=5, presort='auto',
              random_state=100, subsample=1.0, verbose=0, warm_start=False)]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            

### Prediction using Voting on test data

In [31]:
df_test=pd.read_csv('test.csv')
df_test= pd.get_dummies(df_test,columns=['color'])
df_test.head()
X_test =df_test[[ 'bone_length', 'rotting_flesh', 'hair_length', 'has_soul',
       'color_black', 'color_blood', 'color_blue', 'color_clear',
       'color_green', 'color_white']]

In [33]:
df_test['type_int']=grid_stack_estimator.predict(X_test)

In [34]:
df_test.head()

,id,bone_length,rotting_flesh,hair_length,has_soul,color_black,color_blood,color_blue,color_clear,color_green,color_white,type_int
0,3,0.471774,0.387937,0.706087,0.698537,1,0,0,0,0,0,1
1,6,0.427332,0.645024,0.565558,0.451462,0,0,0,0,0,1,2
2,9,0.549602,0.491931,0.660387,0.449809,1,0,0,0,0,0,1
3,10,0.638095,0.682867,0.471409,0.356924,0,0,0,0,0,1,3
4,13,0.361762,0.583997,0.377256,0.276364,1,0,0,0,0,0,3


In [35]:
for i,row in df_test.iterrows():
    #print(i == 'Ghoul')
    #print(i)
    #print(row['type'])
    type_res=''
    if row['type_int'] == 1:
        type_res='Ghoul'
    elif row['type_int'] == 2:
        type_res='Goblin'
    else:
        type_res='Ghost'
    df_test.at[i,'type'] = type_res

In [36]:
df_test.to_csv("Submission_GGG_stack_estimator.csv",columns=['id','type'],index=False)


####  Best ~ 0.703504  train_data score ~ 0.989218 (Ada Boosting with Random Forest)  Kaggle: 0.70321
